# Extracción de Entidades Oficiales: Leyes y Organizaciones Gubernamentales

Este notebook extrae **SOLO** menciones oficiales de:
- Leyes federales de México (patrones precisos)
- Leyes y códigos de la Ciudad de México (patrones precisos)
- Entidades gubernamentales oficiales de CDMX

**Nota**: Solo se utilizan los patrones definidos en los archivos de regex (.py). No se incluyen patrones genéricos.


In [1]:
# Importar librerías
import pandas as pd
import sys

# Importar patrones de regex desde la carpeta regex
sys.path.append('../regex')
from cdmx_entity_patterns_fixed import GOV_ENTITY_REGEX
from federal_laws_patterns_precise import LAWS_REGEX as FEDERAL_LAWS_REGEX
from cdmx_laws_patterns_precise import LAWS_REGEX as CDMX_LAWS_REGEX

# Importar funciones de extracción
sys.path.append('../functions')
from entity_extraction_functions import extract_official_entities, extract_official_entities_parallel


## 1. Cargar datos procesados

In [2]:
# Cargar el archivo generado en el paso anterior (hash_creation.ipynb)
df = pd.read_csv('../../data/03_extracted/legal_documents_with_ids.csv')
df = df[~df['art_id'].str.contains('_TRANS', na=False)]

print(f"Total de artículos cargados: {len(df)}")
print(f"\nColumnas disponibles: {list(df.columns)}")
df.head()


Total de artículos cargados: 30044

Columnas disponibles: ['art_id', 'doc_id', 'Archivo', 'document_name', 'article_name', 'art_num', 'text']


,art_id,doc_id,Archivo,document_name,article_name,art_num,text
0,6D0C4493_1,6D0C4493,RGTO_DE_LA_LEY_DE_CENTROS_PENITENCIARIOS_DE_LA...,Reglamento de la Ley de Centros Penitenciarios...,Artículo 1,1,Las disposiciones contenidas en este Reglament...
1,6D0C4493_2,6D0C4493,RGTO_DE_LA_LEY_DE_CENTROS_PENITENCIARIOS_DE_LA...,Reglamento de la Ley de Centros Penitenciarios...,Artículo 2,2,La aplicación del presente Reglamento correspo...
2,6D0C4493_3,6D0C4493,RGTO_DE_LA_LEY_DE_CENTROS_PENITENCIARIOS_DE_LA...,Reglamento de la Ley de Centros Penitenciarios...,Artículo 3,3,"Para los efectos del presente Reglamento, adem..."
3,6D0C4493_4,6D0C4493,RGTO_DE_LA_LEY_DE_CENTROS_PENITENCIARIOS_DE_LA...,Reglamento de la Ley de Centros Penitenciarios...,Artículo 4,4,Sin perjuicio de los principios que prevén los...
4,6D0C4493_5,6D0C4493,RGTO_DE_LA_LEY_DE_CENTROS_PENITENCIARIOS_DE_LA...,Reglamento de la Ley de Centros Penitenciarios...,Artículo 5,5,El tratamiento para la persona privada de su l...


## 2. Ejecutar extracción de entidades oficiales

### Procesamiento Paralelo

Este notebook utiliza **procesamiento paralelo** para acelerar la extracción de entidades. 

**Ventajas:**
- Divide el trabajo entre múltiples procesos (cores del CPU)
- Puede ser hasta N veces más rápido (donde N = número de cores)
- Ideal para procesar grandes volúmenes de documentos

**Cómo funciona:**
1. El DataFrame se divide en chunks (uno por proceso)
2. Cada chunk se procesa de forma independiente en paralelo
3. Los resultados se combinan al final

**Configuración:**
- `n_jobs=None`: Usa todos los cores disponibles (recomendado)
- `n_jobs=4`: Usa 4 procesos específicamente
- `n_jobs=1`: Equivalente al procesamiento secuencial


In [3]:
# Ejecutar extracción de entidades oficiales con procesamiento PARALELO
# n_jobs=None usa todos los cores disponibles en el sistema
entities_df = extract_official_entities_parallel(
    df=df,
    federal_laws_regex=FEDERAL_LAWS_REGEX,
    cdmx_laws_regex=CDMX_LAWS_REGEX,
    gov_entity_regex=GOV_ENTITY_REGEX,
    n_jobs=None  # Usa todos los cores disponibles
)

print(f"\nRESUMEN FINAL: Se encontraron {len(entities_df)} entidades oficiales")


Iniciando extracción de entidades OFICIALES (PARALELA) a las 00:26:19
Cargando patrones oficiales...
Usando 8 procesos paralelos para 30044 documentos
Procesando 8 chunks en paralelo...


/opt/anaconda3/envs/cdmx_kg/lib/python3.10/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


EXTRACCIÓN COMPLETADA a las 00:37:11
Tiempo total de procesamiento: 652.24 segundos (10.87 minutos)
Documentos procesados: 30044
Total de entidades oficiales extraídas: 13196
Velocidad: 46.1 documentos/segundo

RESUMEN FINAL: Se encontraron 13196 entidades oficiales


## 3. Mostrar estadísticas básicas

In [4]:
# Mostrar estadísticas básicas
print("\nDistribución de entidades por grupo de patrones:")
print(entities_df['pattern_group'].value_counts())

print("\nDistribución de entidades por etiqueta:")
print(entities_df['entity_label'].value_counts())

print("\nPrimeras filas del resultado:")
entities_df.head()



Distribución de entidades por grupo de patrones:
pattern_group
CDMX_OFFICIAL    9615
CDMX_LAWS        2257
FEDERAL_LAWS     1324
Name: count, dtype: int64

Distribución de entidades por etiqueta:
entity_label
CDMX_PODER_EJECUTIVO      5359
CDMX_ORGANOS_AUTONOMOS    2075
CDMX_OTRO                 1786
LEY_CDMX                  1014
CONSTITUCION               671
CDMX_PODER_LEGISLATIVO     381
REGLAMENTO                 365
LEY_GENERAL                332
CODIGO_CDMX                318
LEY_ORGANICA               205
LEY                        197
CODIGO_FEDERAL             160
LEY_FEDERAL                152
CODIGO                     137
LEY_NACIONAL                29
CDMX_SEGURIDAD_SOCIAL       14
OTRO                         1
Name: count, dtype: int64

Primeras filas del resultado:


,doc_id,art_id,document_name,article_name,entity_text,entity_label,pattern_group,before_context,after_context,full_context,words_before_count,words_after_count
0,6D0C4493,6D0C4493_2,Reglamento de la Ley de Centros Penitenciarios...,Artículo 2,Secretaría de Seguridad Ciudadana,CDMX_PODER_EJECUTIVO,CDMX_OFFICIAL,La aplicación del presente Reglamento correspo...,"de la Ciudad de México , a través de la Subsec...",La aplicación del presente Reglamento correspo...,8,30
1,6D0C4493,6D0C4493_2,Reglamento de la Ley de Centros Penitenciarios...,Artículo 2,Ley Nacional de Ejecución Penal,LEY_NACIONAL,FEDERAL_LAWS,"adscrito a los Centros Penitenciarios , Centro...",", la Ley de Centros Penitenciarios de la Ciuda...","adscrito a los Centros Penitenciarios , Centro...",30,16
2,6D0C4493,6D0C4493_2,Reglamento de la Ley de Centros Penitenciarios...,Artículo 2,Ley de Centros Penitenciarios de la Ciudad de ...,LEY_CDMX,CDMX_LAWS,de Sanciones Administrativas y de Integración ...,y demás normativa aplicable .,de Sanciones Administrativas y de Integración ...,30,5
3,6D0C4493,6D0C4493_3,Reglamento de la Ley de Centros Penitenciarios...,Artículo 3,Ley de Centros Penitenciarios de la Ciudad de ...,LEY_CDMX,CDMX_LAWS,"Para los efectos del presente Reglamento , ade...",", se entenderá por : CDUDT : Centro de Diagnós...","Para los efectos del presente Reglamento , ade...",14,30
4,6D0C4493,6D0C4493_3,Reglamento de la Ley de Centros Penitenciarios...,Artículo 3,Constitución Política de los Estados Unidos Me...,CONSTITUCION,FEDERAL_LAWS,"de sanciones penales , de reinserción psicosoc...",; Constitución Local : Constitución Política d...,"de sanciones penales , de reinserción psicosoc...",30,30


## 4. Guardar resultados

In [5]:
# Reorganizar columnas (estandarizadas para todos los extractores)
columns_order = [
    'doc_id', 
    'art_id', 
    'entity_text', 
    'entity_label', 'pattern_group',
    'full_context',
    'words_before_count', 'words_after_count'
]

entities_df = entities_df[columns_order]
entities_df.head()



,doc_id,art_id,entity_text,entity_label,pattern_group,full_context,words_before_count,words_after_count
0,6D0C4493,6D0C4493_2,Secretaría de Seguridad Ciudadana,CDMX_PODER_EJECUTIVO,CDMX_OFFICIAL,La aplicación del presente Reglamento correspo...,8,30
1,6D0C4493,6D0C4493_2,Ley Nacional de Ejecución Penal,LEY_NACIONAL,FEDERAL_LAWS,"adscrito a los Centros Penitenciarios , Centro...",30,16
2,6D0C4493,6D0C4493_2,Ley de Centros Penitenciarios de la Ciudad de ...,LEY_CDMX,CDMX_LAWS,de Sanciones Administrativas y de Integración ...,30,5
3,6D0C4493,6D0C4493_3,Ley de Centros Penitenciarios de la Ciudad de ...,LEY_CDMX,CDMX_LAWS,"Para los efectos del presente Reglamento , ade...",14,30
4,6D0C4493,6D0C4493_3,Constitución Política de los Estados Unidos Me...,CONSTITUCION,FEDERAL_LAWS,"de sanciones penales , de reinserción psicosoc...",30,30


In [6]:
# Guardar resultado completo
entities_output = '../../data/03_extracted/entities_extracted_official.csv'
entities_df.to_csv(entities_output, index=False, encoding='utf-8-sig')
